# 🔁 Model Versioning & Experiment Tracking with MLflow

In [ ]:
# Install MLflow & pyngrok
!pip install mlflow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.2/676.2 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 11.7 MB/s eta 0:00:00


In [ ]:
#download data from kaggle
!pip install opendatasets

In [ ]:
import pandas as pd
import numpy as np
import opendatasets as od
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import joblib

In [ ]:
od.download("https://www.kaggle.com/datasets/taweilo/loan-approval-classification-data?select=loan_data.csv")
#hemantk777  721b095dacb7cfa6129768282921b8fb

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: hemantk777
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/taweilo/loan-approval-classification-data


100%|██████████| 751k/751k [00:00<00:00, 598MB/s]

In [ ]:
df=pd.read_csv("/content/loan-approval-classification-data/loan_data.csv")

In [ ]:
df.head()

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1


In [ ]:
# Example Setup with Logistic Regression (Loan Dataset)
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Load data
df
df = pd.get_dummies(df, drop_first=True)     # categorical

X_1 = df.drop('loan_status', axis=1)
y_1 = df['loan_status']
x_train, x_test, y_train, y_test = train_test_split(X_1, y_1, test_size=0.2, stratify=y_1, random_state=42)

# Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)

In [ ]:
# Set MLflow tracking to local folder
mlflow.set_tracking_uri("file:/content/mlruns")
mlflow.set_experiment("loan_approval_experiment")


2025/07/23 18:22:11 INFO mlflow.tracking.fluent: Experiment with name 'loan_approval_experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///content/mlruns/229885467265834607', creation_time=1753294931659, experiment_id='229885467265834607', last_update_time=1753294931659, lifecycle_stage='active', name='loan_approval_experiment', tags={}>

In [ ]:
# End any previous run if still active
mlflow.end_run()

In [ ]:
# Train + Log with MLflow
with mlflow.start_run(run_name="RandomForest"):
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)
    acc = accuracy_score(y_test, y_pred)

    mlflow.log_param("model_type", "RandomForest")
    mlflow.log_param("n_estimators", 100)
    mlflow.log_metric("accuracy", acc)

    mlflow.sklearn.log_model(model, "model")

    print(f"✅ Model trained. Accuracy: {acc:.4f}")


2025/07/23 18:39:50 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/23 18:39:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Model trained. Accuracy: 0.9286


In [ ]:
# try model 2
with mlflow.start_run(run_name="LogisticRegression"):
  model = LogisticRegression()
  model.fit(x_train, y_train)
  y_pred = model.predict(x_test)
  acc = accuracy_score(y_test, y_pred)
  mlflow.log_param("model_type", "LogisticRegression")
  mlflow.log_metric("accuracy", acc)
  mlflow.sklearn.log_model(model, "model")
  print(f"✅ Model trained. Accuracy: {acc:.4f}")

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

2025/07/23 18:41:38 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/23 18:41:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Model trained. Accuracy: 0.8229


In [ ]:
# ✅ View results from MLflow experiments
experiment = mlflow.get_experiment_by_name("loan_approval_experiment")
runs_df = mlflow.search_runs(experiment_ids=[experiment.experiment_id])

# Show key results
runs_df[['run_id', 'params.model_type', 'params.n_estimators', 'metrics.accuracy']]


,run_id,params.model_type,params.n_estimators,metrics.accuracy
0,6b6417bfe72a4bfc9f5b3869b14c382a,LogisticRegression,None,0.822889
1,ddf78fc488014094943e3174d64d88bc,LogisticRegression,None,0.928556
2,f4cbc69900cb4d5f8cbc64bc9761b54e,RandomForest,100,0.928556
3,feb11ed8c1c34f2f97b26b0bf6fbc0f0,LogisticRegression,None,0.928556
4,f4833e2124634686bedd6ef1a6253574,RandomForest,100,0.928556
5,7008c4a858be4d7eb1e7dcc4753b6feb,None,None,NaN
6,f6145cfae7e645d482e5be19e54bf420,None,None,NaN


In [ ]:
#sort the top runs
runs_df.sort_values("metrics.accuracy", ascending=False)

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.accuracy,params.model_type,params.n_estimators,tags.mlflow.runName,tags.mlflow.source.name,tags.mlflow.source.type,tags.mlflow.user
1,ddf78fc488014094943e3174d64d88bc,229885467265834607,FINISHED,file:///content/mlruns/229885467265834607/ddf7...,2025-07-23 18:39:54.048000+00:00,2025-07-23 18:40:00.418000+00:00,0.928556,LogisticRegression,None,LogisticRegression,/usr/local/lib/python3.11/dist-packages/colab_...,LOCAL,root
3,feb11ed8c1c34f2f97b26b0bf6fbc0f0,229885467265834607,FINISHED,file:///content/mlruns/229885467265834607/feb1...,2025-07-23 18:30:13.165000+00:00,2025-07-23 18:39:43.777000+00:00,0.928556,LogisticRegression,None,illustrious-bug-735,/usr/local/lib/python3.11/dist-packages/colab_...,LOCAL,root
2,f4cbc69900cb4d5f8cbc64bc9761b54e,229885467265834607,FINISHED,file:///content/mlruns/229885467265834607/f4cb...,2025-07-23 18:39:45.558000+00:00,2025-07-23 18:39:54.031000+00:00,0.928556,RandomForest,100,RandomForest,/usr/local/lib/python3.11/dist-packages/colab_...,LOCAL,root
4,f4833e2124634686bedd6ef1a6253574,229885467265834607,FINISHED,file:///content/mlruns/229885467265834607/f483...,2025-07-23 18:23:27.248000+00:00,2025-07-23 18:23:35.876000+00:00,0.928556,RandomForest,100,nervous-fowl-431,/usr/local/lib/python3.11/dist-packages/colab_...,LOCAL,root
0,6b6417bfe72a4bfc9f5b3869b14c382a,229885467265834607,FINISHED,file:///content/mlruns/229885467265834607/6b64...,2025-07-23 18:41:35.930000+00:00,2025-07-23 18:41:42.370000+00:00,0.822889,LogisticRegression,None,LogisticRegression,/usr/local/lib/python3.11/dist-packages/colab_...,LOCAL,root
5,7008c4a858be4d7eb1e7dcc4753b6feb,229885467265834607,FAILED,file:///content/mlruns/229885467265834607/7008...,2025-07-23 18:23:19.274000+00:00,2025-07-23 18:23:25.228000+00:00,NaN,None,None,adaptable-fox-584,/usr/local/lib/python3.11/dist-packages/colab_...,LOCAL,root
6,f6145cfae7e645d482e5be19e54bf420,229885467265834607,FAILED,file:///content/mlruns/229885467265834607/f614...,2025-07-23 18:22:48.385000+00:00,2025-07-23 18:22:48.396000+00:00,NaN,None,None,whimsical-finch-729,/usr/local/lib/python3.11/dist-packages/colab_...,LOCAL,root


In [ ]:
# ✅ Export MLflow results to CSV
runs_df.to_csv("/content/mlflow_results.csv", index=False)